In [4]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer

In [61]:
data = pd.read_csv('train-ch.csv', index_col=0)
data = data.iloc[:, 1:]
data.head()

,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,1,2,3,4,5,6,7,8,9,10,new_feat
ID,,,,,,,,,,,,,,,,,,,
0,Острый ларингофарингит,29,2,NaN,NaN,NaN,На жевательной поверхности 2.6- кариозная поло...,1,Острый ларингофарингит,J06,Острые инфекции верхних дыхательных путей множ...,J00-J06,Острые респираторные инфекции верхних дыхатель...,J00-J99,Болезни органов дыхания,J00-J99,Болезни органов дыхания,J06.0,False
1,Подострый и хронический вагинит,45,2,NaN,NaN,считает себя больной на протяжении многих лет....,NaN,1,Подострый и хронический вагинит,N76,Другие воспалительные болезни влагалища и вульвы,N70-N77,Воспалительные болезни женских тазовых органов,N00-N99,Болезни мочеполовой системы,N00-N99,Болезни мочеполовой системы,N76.1,True
2,"Аллергический контактный дерматит, причина не ...",63,2,NaN,NaN,"Болен около 2-х дней, когда появились описанны...",NaN,1,"Аллергический контактный дерматит, причина не ...",L23,Аллергический контактный дерматит,L20-L30,Дерматит и экзема,L00-L99,Болезни кожи и подкожной клетчатки,L00-L99,Болезни кожи и подкожной клетчатки,L23.9,False
3,Баланопостит,43,2,NaN,не отягощен,NaN,NaN,1,Баланопостит,N48,Другие болезни полового члена,N40-N51,Болезни мужских половых органов,N00-N99,Болезни мочеполовой системы,N00-N99,Болезни мочеполовой системы,N48.1,False
4,Общий медицинский осмотр,29,1,NaN,NaN,провела полное обследование для оперативного л...,NaN,1,Общий медицинский осмотр,Z00,"Общий осмотр и обследование лиц, не имеющих жа...",Z00-Z13,Обращения в учреждения здравоохранения для мед...,Z00-Z99,"Факторы, влияющие на состояние здоровья населе...",Z00-Z99,"Факторы, влияющие на состояние здоровья населе...",Z00.0,False


In [33]:
def calc_bags(data, col_name, min_df=0.05):
    str_anamnez = data[~pd.isnull(data.loc[:, col_name])].loc[:, col_name]
    stemmer = RussianStemmer(ignore_stopwords=True)
    analyzer = CountVectorizer().build_analyzer()
    def stemmed_words(doc):
        return (stemmer.stem(w) for w in analyzer(doc))
    vec = CountVectorizer(tokenizer=stemmed_words, min_df=min_df)
    bag_of_words = vec.fit_transform(str_anamnez)
    return pd.DataFrame(np.asarray(bag_of_words.todense()), columns = list(map(lambda x: x + col_name, vec.get_feature_names())))

In [34]:
new_array = calc_bags(data, '1')

In [35]:
new_array.head()

,болезн1,вагин1,друг1,неуточнен1,остр1,хроническ1
0,0,0,0,0,1,0
1,0,1,0,0,0,1
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [36]:
new_array.shape

(283086, 6)

In [37]:
new_array2 = calc_bags(data, '3')

In [38]:
new_array2.head()

,болезн3,влагалищ3,воспалительн3,вульв3,дорсалг3,друг3,классифицирова3,не3,остр3,поражен3,рубрик3
0,0,0,0,0,0,0,0,0,1,0,0
1,1,1,1,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0


In [39]:
new_array2.shape

(283086, 11)

In [40]:
new_array3 = calc_bags(data, '5', 0.1)
new_array3.head()

,болезн5,воспалительн5,дорсопат5,друг5,женск5,орган5,пут5,тазов5
0,0,0,0,0,0,0,1,0
1,1,1,0,0,1,1,0,1
2,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0


In [41]:
new_array3.shape

(283086, 8)

In [43]:
new_array4 = calc_bags(data, '7', 0.15)
new_array4.head()

,болезн7,мочеполов7,орган7,систем7
0,1,0,1,0
1,1,1,0,1
2,1,0,0,0
3,1,1,0,1
4,0,0,0,0


In [45]:
new_array4.shape

(283086, 4)

In [46]:
new_array5 = calc_bags(data, '9', 0.15)
new_array5.head()

,болезн9,костн9,мочеполов9,мышечн9,орган9,систем9,соединительн9,ткан9
0,1,0,0,0,1,0,0,0
1,1,0,1,0,0,1,0,0
2,1,0,0,0,0,0,0,0
3,1,0,1,0,0,1,0,0
4,0,0,0,0,0,0,0,0


In [47]:
new_array5.shape

(283086, 8)

In [48]:
new_array6 = calc_bags(data, 'Анамнез заболевания', 0.15)
new_array6.head()

,беспокоАнамнез заболевания,болАнамнез заболевания,жалобАнамнез заболевания,леченАнамнез заболевания,наАнамнез заболевания,неАнамнез заболевания,обследованАнамнез заболевания,отмечаАнамнез заболевания,поАнамнез заболевания,теченАнамнез заболевания
0,0,2,0,0,1,0,0,0,1,0
1,0,1,1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0


In [49]:
new_array6.shape

(203656, 10)

In [147]:
new_data_init = pd.concat([data, new_array, new_array2, new_array3, new_array4, new_array5, new_array6], axis=1)
new_data_init.head()

,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,1,2,...,ткан9,беспокоАнамнез заболевания,болАнамнез заболевания,жалобАнамнез заболевания,леченАнамнез заболевания,наАнамнез заболевания,неАнамнез заболевания,обследованАнамнез заболевания,поАнамнез заболевания,теченАнамнез заболевания
0,Острый ларингофарингит,29,2,0,0,0,На жевательной поверхности 2.6- кариозная поло...,1,Острый ларингофарингит,J06,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Подострый и хронический вагинит,45,2,0,0,считает себя больной на протяжении многих лет....,0,1,Подострый и хронический вагинит,N76,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,"Аллергический контактный дерматит, причина не ...",63,2,0,0,"Болен около 2-х дней, когда появились описанны...",0,1,"Аллергический контактный дерматит, причина не ...",L23,...,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
3,Баланопостит,43,2,0,не отягощен,0,0,1,Баланопостит,N48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Общий медицинский осмотр,29,1,0,0,провела полное обследование для оперативного л...,0,1,Общий медицинский осмотр,Z00,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
nastya_data = pd.read_csv('train-exp.csv', index_col=0)
nastya_data.head()

,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,Сост удовл,...,Дизурч явл отр,ПЖК,Сознание,Положение,Тургор,Щитовидка,Дыхание,Боли,Физ отпр,Стеник
ID,,,,,,,,,,,,,,,,,,,,,
0,J06.0,Острый ларингофарингит,29,2,NaN,NaN,NaN,На жевательной поверхности 2.6- кариозная поло...,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,N76.1,Подострый и хронический вагинит,45,2,NaN,NaN,считает себя больной на протяжении многих лет....,NaN,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,NaN,NaN,"Болен около 2-х дней, когда появились описанны...",NaN,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,N48.1,Баланопостит,43,2,NaN,не отягощен,NaN,NaN,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,Z00.0,Общий медицинский осмотр,29,1,NaN,NaN,провела полное обследование для оперативного л...,NaN,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [149]:
nastya_test = pd.read_csv('test-exp.csv', index_col=0)
nastya_test.head()

,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Сост удовл,Лимф узлы,...,Дизурч явл отр,ПЖК,Сознание,Положение,Тургор,Щитовидка,Дыхание,Боли,Физ отпр,Стеник
ID,,,,,,,,,,,,,,,,,,,,,
0,Z39.2,Рутинное послеродовое наблюдение,41,2,Общее самочувствие удовлетворительное. Кожные ...,Аллергию на лекарства - отрицает.,NaN,NaN,True,-1,...,-1,-1,-1,False,-1,False,-1,-1,1,0
1,M54.2,Цервикалгия,64,2,NaN,не отягощен,NaN,NaN,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,M41,Сколиоз,34,2,"Удовлетворительное. Температура тела - 36,6. С...",NaN,NaN,NaN,True,1,...,1,1,1,True,-1,False,-1,-1,1,0
3,J31.2,Хронический фарингит,58,2,NaN,NaN,NaN,Общее состояние удовлетворительное. Конфигурац...,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,M21.4,Плоская стопа [pes planus] (приобретенная),34,2,NaN,NaN,Жалобы сегодня с утра,NaN,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [150]:
nadya_data = pd.read_csv('nadya_col_train.csv', index_col=0)
nadya_data.columns = list(map(lambda x: 'nadya_' + x, nadya_data.columns[:5])) + list(nadya_data.columns[5:])
nadya_data.head()

,nadya_0,nadya_1,nadya_2,nadya_3,nadya_4,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
ID,,,,,,,,,,,,,,
0,-1.0,-1.0,-1.0,-1.0,-1.0,J06.0,Острый ларингофарингит,29,2,0,0,0,На жевательной поверхности 2.6- кариозная поло...,1
1,1.0,0.0,0.0,0.0,0.0,N76.1,Подострый и хронический вагинит,45,2,0,0,считает себя больной на протяжении многих лет....,0,1
2,0.0,0.0,0.0,0.0,0.0,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,0,0,"Болен около 2-х дней, когда появились описанны...",0,1
3,-1.0,-1.0,-1.0,-1.0,-1.0,N48.1,Баланопостит,43,2,0,не отягощен,0,0,1
4,0.0,0.0,0.0,0.0,0.0,Z00.0,Общий медицинский осмотр,29,1,0,0,провела полное обследование для оперативного л...,0,1


In [ ]:
nadya_test = pd.read_csv('nadya_col_test.csv', index_col=0)
nadya_test.columns = list(map(lambda x: 'nadya_' + x, nadya_test.columns[:5])) + list(nadya_test.columns[5:])
nadya_test.head()

In [152]:
new_data = pd.concat([new_data_init, nastya_data.iloc[:, 8:], nadya_data.iloc[:, :5]]).fillna(0)
new_data.head()

,1,10,2,3,4,5,6,7,8,9,...,при1,при3,рубрик1,рубрик3,систем7,систем9,соединительн9,теченАнамнез заболевания,ткан9,хроническ1
0,Острый ларингофарингит,J06.0,J06,Острые инфекции верхних дыхательных путей множ...,J00-J06,Острые респираторные инфекции верхних дыхатель...,J00-J99,Болезни органов дыхания,J00-J99,Болезни органов дыхания,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Подострый и хронический вагинит,N76.1,N76,Другие воспалительные болезни влагалища и вульвы,N70-N77,Воспалительные болезни женских тазовых органов,N00-N99,Болезни мочеполовой системы,N00-N99,Болезни мочеполовой системы,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,"Аллергический контактный дерматит, причина не ...",L23.9,L23,Аллергический контактный дерматит,L20-L30,Дерматит и экзема,L00-L99,Болезни кожи и подкожной клетчатки,L00-L99,Болезни кожи и подкожной клетчатки,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
3,Баланопостит,N48.1,N48,Другие болезни полового члена,N40-N51,Болезни мужских половых органов,N00-N99,Болезни мочеполовой системы,N00-N99,Болезни мочеполовой системы,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Общий медицинский осмотр,Z00.0,Z00,"Общий осмотр и обследование лиц, не имеющих жа...",Z00-Z13,Обращения в учреждения здравоохранения для мед...,Z00-Z99,"Факторы, влияющие на состояние здоровья населе...",Z00-Z99,"Факторы, влияющие на состояние здоровья населе...",...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [153]:
new_data.columns

Index(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'Revisit', 'nadya_0',
       'nadya_1', 'nadya_2', 'nadya_3', 'nadya_4', 'new_feat',
       'Анамнез заболевания', 'Боли', 'Внешний осмотр', 'Возраст', 'Диагноз',
       'Дизурч явл отр', 'Дыхание', 'Живот', 'Кожн покр', 'Лимф узлы',
       'Общее состояние', 'Отеков нет', 'ПЖК', 'Пол', 'Положение', 'Сознание',
       'Сост удовл', 'Стеник', 'Тургор', 'Физ отпр', 'Щитовидка', 'Язык',
       'аллергия', 'беспокоАнамнез заболевания', 'болАнамнез заболевания',
       'болезн1', 'болезн3', 'болезн5', 'болезн7', 'болезн9', 'вагин1',
       'влагалищ3', 'воспален3', 'воспалительн3', 'воспалительн5', 'вульв3',
       'дорсалг3', 'дорсопат5', 'друг1', 'друг3', 'друг5',
       'жалобАнамнез заболевания', 'женск5', 'классифицирова1',
       'классифицирова3', 'костн9', 'леченАнамнез заболевания', 'мочеполов7',
       'мочеполов9', 'мышечн9', 'наАнамнез заболевания', 'нарушен3', 'не3',
       'неАнамнез заболевания', 'неуточнен1', 'обследо

In [154]:
def get_Xy(data):
    X_init = data.loc[:, data.columns != 'Revisit']
    y_init = data.loc[:, 'Revisit']
    size_0 = y_init[y_init == 0].shape
    new_index = list(np.random.choice(list(y_init[y_init == 1].index), replace=False, size=size_0)) + list(y_init[y_init == 0].index)
    X = X_init.loc[new_index]
    y = y_init.loc[new_index]
    X_train, X_test, y_train, y_test = train_test_split(X_init, y_init, test_size=0.2)
    return X, y, X_train, X_test, y_train, y_test

In [155]:
X, y, X_train, X_test, y_train, y_test = get_Xy(new_data)

In [156]:
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(learning_rate=0.02, iterations=3000)

In [166]:
cat_features=list(range(19)) + list(range(20, new_data.shape[1] - 1))

In [103]:
catboost.fit(X=X_train, y=y_train, eval_set=[X_test, y_test], cat_features=cat_features)

0:	learn: 0.6808806	test: 0.6807649	best: 0.6807649 (0)	total: 357ms	remaining: 17m 49s
1:	learn: 0.6692029	test: 0.6689712	best: 0.6689712 (1)	total: 588ms	remaining: 14m 41s
2:	learn: 0.6580636	test: 0.6577426	best: 0.6577426 (2)	total: 793ms	remaining: 13m 11s
3:	learn: 0.6474426	test: 0.6470745	best: 0.6470745 (3)	total: 1.04s	remaining: 12m 59s
4:	learn: 0.6373354	test: 0.6369397	best: 0.6369397 (4)	total: 1.26s	remaining: 12m 35s
5:	learn: 0.6277103	test: 0.6272625	best: 0.6272625 (5)	total: 1.5s	remaining: 12m 29s
6:	learn: 0.6185424	test: 0.6180463	best: 0.6180463 (6)	total: 1.73s	remaining: 12m 18s
7:	learn: 0.6096291	test: 0.6089964	best: 0.6089964 (7)	total: 1.94s	remaining: 12m 6s
8:	learn: 0.6011294	test: 0.6004304	best: 0.6004304 (8)	total: 2.2s	remaining: 12m 11s
9:	learn: 0.5930711	test: 0.5922639	best: 0.5922639 (9)	total: 2.43s	remaining: 12m 7s
10:	learn: 0.5854988	test: 0.5845718	best: 0.5845718 (10)	total: 2.65s	remaining: 12m
11:	learn: 0.5781887	test: 0.5771617	b

92:	learn: 0.4212625	test: 0.4187748	best: 0.4187748 (92)	total: 24.5s	remaining: 12m 46s
93:	learn: 0.4209483	test: 0.4184815	best: 0.4184815 (93)	total: 24.8s	remaining: 12m 46s
94:	learn: 0.4207362	test: 0.4182736	best: 0.4182736 (94)	total: 25s	remaining: 12m 45s
95:	learn: 0.4205644	test: 0.4181066	best: 0.4181066 (95)	total: 25.3s	remaining: 12m 44s
96:	learn: 0.4203944	test: 0.4179444	best: 0.4179444 (96)	total: 25.6s	remaining: 12m 45s
97:	learn: 0.4201380	test: 0.4177009	best: 0.4177009 (97)	total: 25.9s	remaining: 12m 45s
98:	learn: 0.4199705	test: 0.4175298	best: 0.4175298 (98)	total: 26.2s	remaining: 12m 46s
99:	learn: 0.4197814	test: 0.4173404	best: 0.4173404 (99)	total: 26.4s	remaining: 12m 46s
100:	learn: 0.4196094	test: 0.4171660	best: 0.4171660 (100)	total: 26.7s	remaining: 12m 46s
101:	learn: 0.4194588	test: 0.4170152	best: 0.4170152 (101)	total: 27s	remaining: 12m 45s
102:	learn: 0.4192724	test: 0.4168200	best: 0.4168200 (102)	total: 27.2s	remaining: 12m 44s
103:	lea

182:	learn: 0.4131641	test: 0.4111217	best: 0.4111217 (182)	total: 51.2s	remaining: 13m 8s
183:	learn: 0.4131507	test: 0.4111090	best: 0.4111090 (183)	total: 51.5s	remaining: 13m 8s
184:	learn: 0.4131318	test: 0.4110879	best: 0.4110879 (184)	total: 51.8s	remaining: 13m 8s
185:	learn: 0.4130854	test: 0.4110473	best: 0.4110473 (185)	total: 52.2s	remaining: 13m 9s
186:	learn: 0.4130637	test: 0.4110311	best: 0.4110311 (186)	total: 52.5s	remaining: 13m 9s
187:	learn: 0.4130288	test: 0.4110009	best: 0.4110009 (187)	total: 52.8s	remaining: 13m 10s
188:	learn: 0.4129901	test: 0.4109721	best: 0.4109721 (188)	total: 53.2s	remaining: 13m 10s
189:	learn: 0.4129665	test: 0.4109524	best: 0.4109524 (189)	total: 53.5s	remaining: 13m 10s
190:	learn: 0.4129512	test: 0.4109348	best: 0.4109348 (190)	total: 53.7s	remaining: 13m 10s
191:	learn: 0.4129308	test: 0.4109219	best: 0.4109219 (191)	total: 54.1s	remaining: 13m 11s
192:	learn: 0.4129090	test: 0.4109032	best: 0.4109032 (192)	total: 54.4s	remaining: 1

271:	learn: 0.4118521	test: 0.4101637	best: 0.4101637 (271)	total: 1m 19s	remaining: 13m 13s
272:	learn: 0.4118434	test: 0.4101577	best: 0.4101577 (272)	total: 1m 19s	remaining: 13m 13s
273:	learn: 0.4118373	test: 0.4101528	best: 0.4101528 (273)	total: 1m 19s	remaining: 13m 13s
274:	learn: 0.4118273	test: 0.4101487	best: 0.4101487 (274)	total: 1m 20s	remaining: 13m 13s
275:	learn: 0.4118183	test: 0.4101461	best: 0.4101461 (275)	total: 1m 20s	remaining: 13m 13s
276:	learn: 0.4118103	test: 0.4101358	best: 0.4101358 (276)	total: 1m 20s	remaining: 13m 13s
277:	learn: 0.4118037	test: 0.4101339	best: 0.4101339 (277)	total: 1m 21s	remaining: 13m 13s
278:	learn: 0.4117954	test: 0.4101292	best: 0.4101292 (278)	total: 1m 21s	remaining: 13m 13s
279:	learn: 0.4117885	test: 0.4101244	best: 0.4101244 (279)	total: 1m 21s	remaining: 13m 12s
280:	learn: 0.4117830	test: 0.4101221	best: 0.4101221 (280)	total: 1m 21s	remaining: 13m 11s
281:	learn: 0.4117780	test: 0.4101206	best: 0.4101206 (281)	total: 1m 

360:	learn: 0.4111752	test: 0.4098013	best: 0.4098013 (360)	total: 1m 46s	remaining: 12m 57s
361:	learn: 0.4111678	test: 0.4097959	best: 0.4097959 (361)	total: 1m 46s	remaining: 12m 57s
362:	learn: 0.4111589	test: 0.4097900	best: 0.4097900 (362)	total: 1m 47s	remaining: 12m 57s
363:	learn: 0.4111495	test: 0.4097867	best: 0.4097867 (363)	total: 1m 47s	remaining: 12m 57s
364:	learn: 0.4111480	test: 0.4097868	best: 0.4097867 (363)	total: 1m 47s	remaining: 12m 56s
365:	learn: 0.4111425	test: 0.4097837	best: 0.4097837 (365)	total: 1m 47s	remaining: 12m 56s
366:	learn: 0.4111348	test: 0.4097811	best: 0.4097811 (366)	total: 1m 48s	remaining: 12m 56s
367:	learn: 0.4111301	test: 0.4097795	best: 0.4097795 (367)	total: 1m 48s	remaining: 12m 56s
368:	learn: 0.4111107	test: 0.4097677	best: 0.4097677 (368)	total: 1m 48s	remaining: 12m 56s
369:	learn: 0.4111053	test: 0.4097656	best: 0.4097656 (369)	total: 1m 49s	remaining: 12m 56s
370:	learn: 0.4111035	test: 0.4097652	best: 0.4097652 (370)	total: 1m 

...

1160:	learn: 0.4081516	test: 0.4091920	best: 0.4091919 (1158)	total: 5m 51s	remaining: 9m 16s
1161:	learn: 0.4081430	test: 0.4091904	best: 0.4091904 (1161)	total: 5m 51s	remaining: 9m 16s
1162:	learn: 0.4081393	test: 0.4091901	best: 0.4091901 (1162)	total: 5m 52s	remaining: 9m 16s
1163:	learn: 0.4081364	test: 0.4091907	best: 0.4091901 (1162)	total: 5m 52s	remaining: 9m 15s
1164:	learn: 0.4081308	test: 0.4091911	best: 0.4091901 (1162)	total: 5m 52s	remaining: 9m 15s
1165:	learn: 0.4081274	test: 0.4091926	best: 0.4091901 (1162)	total: 5m 52s	remaining: 9m 15s
1166:	learn: 0.4081217	test: 0.4091906	best: 0.4091901 (1162)	total: 5m 53s	remaining: 9m 14s
1167:	learn: 0.4081166	test: 0.4091909	best: 0.4091901 (1162)	total: 5m 53s	remaining: 9m 14s
1168:	learn: 0.4081113	test: 0.4091878	best: 0.4091878 (1168)	total: 5m 53s	remaining: 9m 14s
1169:	learn: 0.4081095	test: 0.4091883	best: 0.4091878 (1168)	total: 5m 54s	remaining: 9m 13s
1170:	learn: 0.4081070	test: 0.4091876	best: 0.4091876 (1170

1249:	learn: 0.4078681	test: 0.4091609	best: 0.4091602 (1242)	total: 6m 19s	remaining: 8m 51s
1250:	learn: 0.4078668	test: 0.4091611	best: 0.4091602 (1242)	total: 6m 19s	remaining: 8m 50s
1251:	learn: 0.4078629	test: 0.4091628	best: 0.4091602 (1242)	total: 6m 19s	remaining: 8m 50s
1252:	learn: 0.4078608	test: 0.4091621	best: 0.4091602 (1242)	total: 6m 20s	remaining: 8m 50s
1253:	learn: 0.4078563	test: 0.4091599	best: 0.4091599 (1253)	total: 6m 20s	remaining: 8m 50s
1254:	learn: 0.4078532	test: 0.4091602	best: 0.4091599 (1253)	total: 6m 21s	remaining: 8m 49s
1255:	learn: 0.4078515	test: 0.4091597	best: 0.4091597 (1255)	total: 6m 21s	remaining: 8m 49s
1256:	learn: 0.4078480	test: 0.4091617	best: 0.4091597 (1255)	total: 6m 21s	remaining: 8m 49s
1257:	learn: 0.4078471	test: 0.4091614	best: 0.4091597 (1255)	total: 6m 21s	remaining: 8m 48s
1258:	learn: 0.4078439	test: 0.4091611	best: 0.4091597 (1255)	total: 6m 22s	remaining: 8m 48s
1259:	learn: 0.4078414	test: 0.4091615	best: 0.4091597 (1255

1337:	learn: 0.4075965	test: 0.4091342	best: 0.4091331 (1336)	total: 6m 46s	remaining: 8m 25s
1338:	learn: 0.4075929	test: 0.4091339	best: 0.4091331 (1336)	total: 6m 46s	remaining: 8m 24s
1339:	learn: 0.4075899	test: 0.4091330	best: 0.4091330 (1339)	total: 6m 47s	remaining: 8m 24s
1340:	learn: 0.4075858	test: 0.4091324	best: 0.4091324 (1340)	total: 6m 47s	remaining: 8m 24s
1341:	learn: 0.4075807	test: 0.4091338	best: 0.4091324 (1340)	total: 6m 47s	remaining: 8m 23s
1342:	learn: 0.4075788	test: 0.4091340	best: 0.4091324 (1340)	total: 6m 47s	remaining: 8m 23s
1343:	learn: 0.4075772	test: 0.4091343	best: 0.4091324 (1340)	total: 6m 48s	remaining: 8m 23s
1344:	learn: 0.4075743	test: 0.4091326	best: 0.4091324 (1340)	total: 6m 48s	remaining: 8m 22s
1345:	learn: 0.4075714	test: 0.4091329	best: 0.4091324 (1340)	total: 6m 48s	remaining: 8m 22s
1346:	learn: 0.4075626	test: 0.4091309	best: 0.4091309 (1346)	total: 6m 49s	remaining: 8m 22s
1347:	learn: 0.4075610	test: 0.4091307	best: 0.4091307 (1347

1426:	learn: 0.4073342	test: 0.4091170	best: 0.4091127 (1402)	total: 7m 13s	remaining: 7m 58s
1427:	learn: 0.4073330	test: 0.4091170	best: 0.4091127 (1402)	total: 7m 14s	remaining: 7m 57s
1428:	learn: 0.4073316	test: 0.4091171	best: 0.4091127 (1402)	total: 7m 14s	remaining: 7m 57s
1429:	learn: 0.4073280	test: 0.4091169	best: 0.4091127 (1402)	total: 7m 14s	remaining: 7m 57s
1430:	learn: 0.4073259	test: 0.4091172	best: 0.4091127 (1402)	total: 7m 15s	remaining: 7m 56s
1431:	learn: 0.4073186	test: 0.4091129	best: 0.4091127 (1402)	total: 7m 15s	remaining: 7m 56s
1432:	learn: 0.4073140	test: 0.4091126	best: 0.4091126 (1432)	total: 7m 15s	remaining: 7m 56s
1433:	learn: 0.4073082	test: 0.4091117	best: 0.4091117 (1433)	total: 7m 15s	remaining: 7m 56s
1434:	learn: 0.4073043	test: 0.4091103	best: 0.4091103 (1434)	total: 7m 16s	remaining: 7m 55s
1435:	learn: 0.4073022	test: 0.4091109	best: 0.4091103 (1434)	total: 7m 16s	remaining: 7m 55s
1436:	learn: 0.4072978	test: 0.4091102	best: 0.4091102 (1436

KeyboardInterrupt: 

In [ ]:
f_imp = catboost.get_feature_importance(X=X, y=y, cat_features=cat_features)

In [ ]:
f_imp

In [108]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, catboost.predict_proba(X)[:, 1], pos_label=1)
metrics.auc(fpr, tpr)

CatboostError: There is no trained model to use predict(). Use fit() to train model. Then use predict().

In [167]:
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(learning_rate=0.015, iterations=3000)

In [ ]:
catboost.fit(X=X, y=y, cat_features=cat_features)

0:	learn: 0.6631444	total: 3.02s	remaining: 2h 31m 11s
1:	learn: 0.6363024	total: 5.61s	remaining: 2h 20m 11s
2:	learn: 0.6098115	total: 8.31s	remaining: 2h 18m 23s
3:	learn: 0.5883880	total: 10.7s	remaining: 2h 13m 39s
4:	learn: 0.5681527	total: 13.2s	remaining: 2h 11m 17s
5:	learn: 0.5493800	total: 15.3s	remaining: 2h 7m 11s
6:	learn: 0.5338207	total: 17.5s	remaining: 2h 4m 27s
7:	learn: 0.5180110	total: 19.5s	remaining: 2h 1m 46s
8:	learn: 0.5043107	total: 21.5s	remaining: 1h 59m 18s
9:	learn: 0.4901540	total: 24s	remaining: 1h 59m 28s
10:	learn: 0.4803340	total: 25.9s	remaining: 1h 57m 30s
11:	learn: 0.4713019	total: 27.9s	remaining: 1h 55m 59s
12:	learn: 0.4625244	total: 30s	remaining: 1h 54m 59s
13:	learn: 0.4525845	total: 32.7s	remaining: 1h 56m 11s
14:	learn: 0.4436870	total: 35.1s	remaining: 1h 56m 33s
15:	learn: 0.4356315	total: 37.8s	remaining: 1h 57m 22s
16:	learn: 0.4285504	total: 40.3s	remaining: 1h 57m 58s
17:	learn: 0.4220597	total: 42.7s	remaining: 1h 57m 56s
18:	learn

145:	learn: 0.2136199	total: 7m 11s	remaining: 2h 20m 25s
146:	learn: 0.2135851	total: 7m 13s	remaining: 2h 20m 21s
147:	learn: 0.2135577	total: 7m 16s	remaining: 2h 20m 13s
148:	learn: 0.2135259	total: 7m 19s	remaining: 2h 20m 10s
149:	learn: 0.2134932	total: 7m 22s	remaining: 2h 20m 12s
150:	learn: 0.2134654	total: 7m 25s	remaining: 2h 20m 7s
151:	learn: 0.2134353	total: 7m 28s	remaining: 2h 20m 3s
152:	learn: 0.2134006	total: 7m 31s	remaining: 2h 20m 6s
153:	learn: 0.2133740	total: 7m 35s	remaining: 2h 20m 10s
154:	learn: 0.2133465	total: 7m 38s	remaining: 2h 20m 13s
155:	learn: 0.2133160	total: 7m 41s	remaining: 2h 20m 18s
156:	learn: 0.2132847	total: 7m 44s	remaining: 2h 20m 19s
157:	learn: 0.2132558	total: 7m 48s	remaining: 2h 20m 25s
158:	learn: 0.2132325	total: 7m 51s	remaining: 2h 20m 21s
159:	learn: 0.2132099	total: 7m 54s	remaining: 2h 20m 25s
160:	learn: 0.2131914	total: 7m 57s	remaining: 2h 20m 19s
161:	learn: 0.2131704	total: 8m	remaining: 2h 20m 22s
162:	learn: 0.2131479

286:	learn: 0.2121429	total: 15m 11s	remaining: 2h 23m 39s
287:	learn: 0.2121404	total: 15m 15s	remaining: 2h 23m 42s
288:	learn: 0.2121391	total: 15m 19s	remaining: 2h 23m 42s
289:	learn: 0.2121378	total: 15m 22s	remaining: 2h 23m 41s
290:	learn: 0.2121364	total: 15m 26s	remaining: 2h 23m 43s
291:	learn: 0.2121353	total: 15m 29s	remaining: 2h 23m 41s
292:	learn: 0.2121334	total: 15m 32s	remaining: 2h 23m 37s
293:	learn: 0.2121317	total: 15m 36s	remaining: 2h 23m 36s
294:	learn: 0.2121296	total: 15m 39s	remaining: 2h 23m 31s
295:	learn: 0.2121280	total: 15m 42s	remaining: 2h 23m 29s
296:	learn: 0.2121247	total: 15m 46s	remaining: 2h 23m 33s
297:	learn: 0.2121198	total: 15m 49s	remaining: 2h 23m 28s
298:	learn: 0.2121189	total: 15m 52s	remaining: 2h 23m 28s
299:	learn: 0.2121172	total: 15m 56s	remaining: 2h 23m 26s
300:	learn: 0.2121149	total: 15m 59s	remaining: 2h 23m 26s
301:	learn: 0.2121136	total: 16m 3s	remaining: 2h 23m 25s
302:	learn: 0.2121128	total: 16m 6s	remaining: 2h 23m 24s

426:	learn: 0.2119495	total: 23m	remaining: 2h 18m 41s
427:	learn: 0.2119493	total: 23m 3s	remaining: 2h 18m 36s
428:	learn: 0.2119486	total: 23m 7s	remaining: 2h 18m 33s
429:	learn: 0.2119484	total: 23m 10s	remaining: 2h 18m 27s
430:	learn: 0.2119455	total: 23m 12s	remaining: 2h 18m 22s
431:	learn: 0.2119433	total: 23m 15s	remaining: 2h 18m 18s
432:	learn: 0.2119415	total: 23m 19s	remaining: 2h 18m 14s
433:	learn: 0.2119408	total: 23m 22s	remaining: 2h 18m 9s
434:	learn: 0.2119401	total: 23m 24s	remaining: 2h 18m 4s
435:	learn: 0.2119399	total: 23m 27s	remaining: 2h 17m 59s
436:	learn: 0.2119397	total: 23m 30s	remaining: 2h 17m 54s
437:	learn: 0.2119394	total: 23m 33s	remaining: 2h 17m 48s
438:	learn: 0.2119392	total: 23m 36s	remaining: 2h 17m 44s
439:	learn: 0.2119389	total: 23m 39s	remaining: 2h 17m 39s
440:	learn: 0.2119387	total: 23m 42s	remaining: 2h 17m 34s
441:	learn: 0.2119373	total: 23m 45s	remaining: 2h 17m 30s
442:	learn: 0.2119372	total: 23m 48s	remaining: 2h 17m 25s
443:	

566:	learn: 0.2117961	total: 30m 10s	remaining: 2h 9m 27s
567:	learn: 0.2117928	total: 30m 13s	remaining: 2h 9m 24s
568:	learn: 0.2117925	total: 30m 16s	remaining: 2h 9m 20s
569:	learn: 0.2117924	total: 30m 19s	remaining: 2h 9m 15s
570:	learn: 0.2117922	total: 30m 22s	remaining: 2h 9m 11s
571:	learn: 0.2117921	total: 30m 25s	remaining: 2h 9m 6s
572:	learn: 0.2117901	total: 30m 27s	remaining: 2h 9m 2s
573:	learn: 0.2117900	total: 30m 30s	remaining: 2h 8m 58s
574:	learn: 0.2117896	total: 30m 34s	remaining: 2h 8m 55s
575:	learn: 0.2117885	total: 30m 37s	remaining: 2h 8m 52s
576:	learn: 0.2117884	total: 30m 40s	remaining: 2h 8m 47s
577:	learn: 0.2117879	total: 30m 43s	remaining: 2h 8m 44s
578:	learn: 0.2117878	total: 30m 46s	remaining: 2h 8m 40s
579:	learn: 0.2117871	total: 30m 49s	remaining: 2h 8m 37s
580:	learn: 0.2117859	total: 30m 52s	remaining: 2h 8m 32s
581:	learn: 0.2117819	total: 30m 55s	remaining: 2h 8m 30s
582:	learn: 0.2117814	total: 30m 58s	remaining: 2h 8m 26s
583:	learn: 0.21

709:	learn: 0.2117104	total: 37m 25s	remaining: 2h 42s
710:	learn: 0.2117103	total: 37m 28s	remaining: 2h 38s
711:	learn: 0.2117103	total: 37m 31s	remaining: 2h 34s
712:	learn: 0.2117102	total: 37m 34s	remaining: 2h 31s
713:	learn: 0.2117100	total: 37m 37s	remaining: 2h 27s
714:	learn: 0.2117099	total: 37m 40s	remaining: 2h 24s
715:	learn: 0.2117099	total: 37m 43s	remaining: 2h 19s
716:	learn: 0.2117070	total: 37m 47s	remaining: 2h 18s
717:	learn: 0.2117070	total: 37m 49s	remaining: 2h 14s
718:	learn: 0.2117066	total: 37m 52s	remaining: 2h 10s
719:	learn: 0.2117060	total: 37m 55s	remaining: 2h 6s
720:	learn: 0.2117057	total: 37m 58s	remaining: 2h 3s
721:	learn: 0.2117046	total: 38m 2s	remaining: 2h
722:	learn: 0.2117044	total: 38m 4s	remaining: 1h 59m 56s
723:	learn: 0.2117016	total: 38m 8s	remaining: 1h 59m 54s
724:	learn: 0.2117014	total: 38m 11s	remaining: 1h 59m 51s
725:	learn: 0.2117013	total: 38m 14s	remaining: 1h 59m 47s
726:	learn: 0.2117008	total: 38m 17s	remaining: 1h 59m 42s

850:	learn: 0.2116438	total: 44m 30s	remaining: 1h 52m 24s
851:	learn: 0.2116417	total: 44m 33s	remaining: 1h 52m 19s
852:	learn: 0.2116412	total: 44m 36s	remaining: 1h 52m 16s
853:	learn: 0.2116409	total: 44m 39s	remaining: 1h 52m 13s
854:	learn: 0.2116409	total: 44m 42s	remaining: 1h 52m 9s
855:	learn: 0.2116407	total: 44m 45s	remaining: 1h 52m 5s
856:	learn: 0.2116406	total: 44m 48s	remaining: 1h 52m 1s
857:	learn: 0.2116406	total: 44m 50s	remaining: 1h 51m 57s
858:	learn: 0.2116404	total: 44m 53s	remaining: 1h 51m 53s
859:	learn: 0.2116403	total: 44m 56s	remaining: 1h 51m 49s
860:	learn: 0.2116402	total: 44m 59s	remaining: 1h 51m 45s
861:	learn: 0.2116401	total: 45m 2s	remaining: 1h 51m 42s
862:	learn: 0.2116399	total: 45m 5s	remaining: 1h 51m 38s
863:	learn: 0.2116398	total: 45m 7s	remaining: 1h 51m 34s
864:	learn: 0.2116397	total: 45m 10s	remaining: 1h 51m 30s
865:	learn: 0.2116392	total: 45m 13s	remaining: 1h 51m 27s
866:	learn: 0.2116391	total: 45m 16s	remaining: 1h 51m 24s
867

990:	learn: 0.2116052	total: 52m 1s	remaining: 1h 45m 28s
991:	learn: 0.2116051	total: 52m 5s	remaining: 1h 45m 27s
992:	learn: 0.2116050	total: 52m 9s	remaining: 1h 45m 24s
993:	learn: 0.2116049	total: 52m 12s	remaining: 1h 45m 21s
994:	learn: 0.2116042	total: 52m 16s	remaining: 1h 45m 20s
995:	learn: 0.2116038	total: 52m 19s	remaining: 1h 45m 17s
996:	learn: 0.2116037	total: 52m 23s	remaining: 1h 45m 15s
997:	learn: 0.2116036	total: 52m 27s	remaining: 1h 45m 13s
998:	learn: 0.2116036	total: 52m 31s	remaining: 1h 45m 11s
999:	learn: 0.2116032	total: 52m 35s	remaining: 1h 45m 10s
1000:	learn: 0.2116029	total: 52m 39s	remaining: 1h 45m 8s
1001:	learn: 0.2116028	total: 52m 43s	remaining: 1h 45m 7s
1002:	learn: 0.2116023	total: 52m 47s	remaining: 1h 45m 5s
1003:	learn: 0.2116022	total: 52m 50s	remaining: 1h 45m 2s
1004:	learn: 0.2116021	total: 52m 53s	remaining: 1h 44m 58s
1005:	learn: 0.2116018	total: 52m 56s	remaining: 1h 44m 56s
1006:	learn: 0.2116018	total: 53m	remaining: 1h 44m 53s
1

1129:	learn: 0.2115705	total: 1h 49s	remaining: 1h 40m 40s
1130:	learn: 0.2115703	total: 1h 53s	remaining: 1h 40m 37s
1131:	learn: 0.2115702	total: 1h 56s	remaining: 1h 40m 34s
1132:	learn: 0.2115698	total: 1h 1m	remaining: 1h 40m 31s
1133:	learn: 0.2115698	total: 1h 1m 4s	remaining: 1h 40m 29s
1134:	learn: 0.2115696	total: 1h 1m 8s	remaining: 1h 40m 27s
1135:	learn: 0.2115694	total: 1h 1m 11s	remaining: 1h 40m 25s
1136:	learn: 0.2115693	total: 1h 1m 15s	remaining: 1h 40m 22s
1137:	learn: 0.2115691	total: 1h 1m 19s	remaining: 1h 40m 20s
1138:	learn: 0.2115691	total: 1h 1m 23s	remaining: 1h 40m 18s
1139:	learn: 0.2115689	total: 1h 1m 27s	remaining: 1h 40m 16s
1140:	learn: 0.2115689	total: 1h 1m 30s	remaining: 1h 40m 13s
1141:	learn: 0.2115687	total: 1h 1m 34s	remaining: 1h 40m 11s
1142:	learn: 0.2115686	total: 1h 1m 38s	remaining: 1h 40m 9s
1143:	learn: 0.2115682	total: 1h 1m 43s	remaining: 1h 40m 7s
1144:	learn: 0.2115680	total: 1h 1m 47s	remaining: 1h 40m 6s
1145:	learn: 0.2115679	tot

1263:	learn: 0.2115391	total: 1h 9m 30s	remaining: 1h 35m 28s
1264:	learn: 0.2115388	total: 1h 9m 34s	remaining: 1h 35m 25s
1265:	learn: 0.2115386	total: 1h 9m 38s	remaining: 1h 35m 23s
1266:	learn: 0.2115385	total: 1h 9m 42s	remaining: 1h 35m 21s
1267:	learn: 0.2115383	total: 1h 9m 46s	remaining: 1h 35m 18s
1268:	learn: 0.2115381	total: 1h 9m 50s	remaining: 1h 35m 16s
1269:	learn: 0.2115380	total: 1h 9m 55s	remaining: 1h 35m 14s
1270:	learn: 0.2115379	total: 1h 9m 59s	remaining: 1h 35m 12s
1271:	learn: 0.2115375	total: 1h 10m 3s	remaining: 1h 35m 10s
1272:	learn: 0.2115375	total: 1h 10m 7s	remaining: 1h 35m 7s
1273:	learn: 0.2115373	total: 1h 10m 11s	remaining: 1h 35m 5s
1274:	learn: 0.2115371	total: 1h 10m 15s	remaining: 1h 35m 2s
1275:	learn: 0.2115367	total: 1h 10m 18s	remaining: 1h 35m
1276:	learn: 0.2115366	total: 1h 10m 23s	remaining: 1h 34m 58s
1277:	learn: 0.2115366	total: 1h 10m 27s	remaining: 1h 34m 55s
1278:	learn: 0.2115343	total: 1h 10m 31s	remaining: 1h 34m 53s
1279:	lea

In [ ]:
real_test = pd.read_csv('test-ch.csv', index_col=0)
real_test = real_test.iloc[:, 1:]
real_test.head()

In [ ]:
new_array = calc_bags(real_test, '1')
new_array2 = calc_bags(real_test, '3')
new_array3 = calc_bags(real_test, '5', 0.1)
new_array4 = calc_bags(real_test, '7', 0.15)
new_array5 = calc_bags(real_test, '9', 0.15)
new_array6 = calc_bags(real_test, 'Анамнез заболевания', 0.15)

In [ ]:
new_test = pd.concat([real_test, new_array, new_array2, new_array3, new_array4, new_array5, new_array6], axis=1)

In [ ]:
new_real_test = pd.concat([new_test, nastya_test.iloc[:, 8:], nadya_test.iloc[:, :5]], axis=1).fillna(0)

In [ ]:
ans = catboost.predict_proba(new_real_test)[:, 1]
ans_pandas = pd.DataFrame(data={'ID': new_real_test.index, 'proba': ans}).reset_index(drop=True)
ans_pandas.to_csv('embed_nkd.csv', index=False)